In [1]:
import os
import json
import boto3

import pandas as pd

from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()

ACCESS_KEY = os.getenv("ID_KEY_ACCESS")
ACCESS_KEY_SECRETE = os.getenv("KEY_SECRET")
BUCKET_NAME = os.getenv("BUCKETNAME")

In [2]:
df_1 = pd.read_json("../hotel_1.json")
df_2 = pd.read_json("../hotel_2.json")
df_3 = pd.read_json("../temps.json")

In [3]:
# merge des 3 dataframe
def merge_outer(data_1:pd.DataFrame, data_2:pd.DataFrame, on:str) -> pd.DataFrame:
    df = pd.merge(data_1, data_2, how='outer', on=on)
    return df

df = merge_outer(df_1, df_2, "hotel")
df = merge_outer(df, df_3, "ville_rechercher")

In [4]:
# drop des hotel ou il manque la note
df.dropna()

,ville_rechercher,ville_précise,hotel,url,note,description,latitutde,longitude,2022-12-05 12:00:00,2022-12-06 12:00:00,2022-12-07 12:00:00,2022-12-08 12:00:00,2022-12-09 12:00:00
0,Le Mont-Saint-Michel,Le Mont-Saint-Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"7,8",Vous pouvez bénéficier d'une réduction Genius ...,48.617587,-1.510396,"{'temperature': 6.5600000000000005, 'temps': '...","{'temperature': 6.73, 'temps': 'nuageux'}","{'temperature': 7.85, 'temps': 'légère pluie'}","{'temperature': 5.52, 'temps': 'partiellement ...","{'temperature': 5.12, 'temps': 'partiellement ..."
1,Le Mont-Saint-Michel,Le Mont-Saint-Michel,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,"7,0",Vous pouvez bénéficier d'une réduction Genius ...,48.635085,-1.510540,"{'temperature': 6.5600000000000005, 'temps': '...","{'temperature': 6.73, 'temps': 'nuageux'}","{'temperature': 7.85, 'temps': 'légère pluie'}","{'temperature': 5.52, 'temps': 'partiellement ...","{'temperature': 5.12, 'temps': 'partiellement ..."
2,Le Mont-Saint-Michel,Le Mont-Saint-Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,"7,1",L'hôtel De La Digue est un établissement tradi...,48.616882,-1.510918,"{'temperature': 6.5600000000000005, 'temps': '...","{'temperature': 6.73, 'temps': 'nuageux'}","{'temperature': 7.85, 'temps': 'légère pluie'}","{'temperature': 5.52, 'temps': 'partiellement ...","{'temperature': 5.12, 'temps': 'partiellement ..."
3,Le Mont-Saint-Michel,Le Mont-Saint-Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,"7,9",Vous pouvez bénéficier d'une réduction Genius ...,48.616263,-1.510906,"{'temperature': 6.5600000000000005, 'temps': '...","{'temperature': 6.73, 'temps': 'nuageux'}","{'temperature': 7.85, 'temps': 'légère pluie'}","{'temperature': 5.52, 'temps': 'partiellement ...","{'temperature': 5.12, 'temps': 'partiellement ..."
4,Le Mont-Saint-Michel,Le Mont-Saint-Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,"7,3",Vous pouvez bénéficier d'une réduction Genius ...,48.635349,-1.510379,"{'temperature': 6.5600000000000005, 'temps': '...","{'temperature': 6.73, 'temps': 'nuageux'}","{'temperature': 7.85, 'temps': 'légère pluie'}","{'temperature': 5.52, 'temps': 'partiellement ...","{'temperature': 5.12, 'temps': 'partiellement ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,Bormes-les-Mimosas,Bormes-les-Mimosas,Asklan,https://www.booking.com/hotel/fr/assklan.fr.ht...,"7,7",Vous pouvez bénéficier d'une réduction Genius ...,43.122648,6.359466,"{'temperature': 13.35, 'temps': 'ciel dégagé'}","{'temperature': 12.27, 'temps': 'couvert'}","{'temperature': 11.79, 'temps': 'ciel dégagé'}","{'temperature': 11.02, 'temps': 'couvert'}","{'temperature': 11, 'temps': 'couvert'}"
873,Bormes-les-Mimosas,Bormes-les-Mimosas,Studio clim wifi centre historique de Bormes l...,https://www.booking.com/hotel/fr/studio-centre...,"8,5",Vous pouvez bénéficier d'une réduction Genius ...,43.150361,6.342626,"{'temperature': 13.35, 'temps': 'ciel dégagé'}","{'temperature': 12.27, 'temps': 'couvert'}","{'temperature': 11.79, 'temps': 'ciel dégagé'}","{'temperature': 11.02, 'temps': 'couvert'}","{'temperature': 11, 'temps': 'couvert'}"
874,Bormes-les-Mimosas,Bormes-les-Mimosas,La petite Provence,https://www.booking.com/hotel/fr/lovely-place-...,"8,6",La petite Provence est située à Bormes-les-Mim...,43.151271,6.341907,"{'temperature': 13.35, 'temps': 'ciel dégagé'}","{'temperature': 12.27, 'temps': 'couvert'}","{'temperature': 11.79, 'temps': 'ciel dégagé'}","{'temperature': 11.02, 'temps': 'couvert'}","{'temperature': 11, 'temps': 'couvert'}"
875,Bormes-les-Mimosas,Bormes-les-Mimosas,Studio avec Jardin et Parking Privés,https://www.booking.com/hotel/fr/217-rue-des-i...,"8,5",Le Studio avec Jardin et Parking Privés est si...,43.140363,6.336715,"{'temperature': 13.35, 'temps': 'ciel dégagé'}","{'temperature': 12.27, 'temps': 'couvert'}","{'temper

In [7]:
# sauvegarde du dataframe en csv pour envoie dans un s3
csv = df.to_csv()
df.to_csv("hotel.csv")

In [2]:
# création d'une session aws
session = boto3.Session(aws_access_key_id=ACCESS_KEY, 
                        aws_secret_access_key=ACCESS_KEY_SECRETE)
s3 = session.resource("s3")

In [ ]:
# création d'un bucket s3
bucket = s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [9]:
# ajout du fichier csv dans le compartiment bucket s3
put_object = bucket.put_object(Key="hotel.csv", Body=csv)

In [ ]:
# recuperation du fichier csv dans un s3
df = pd.read_csv('s3://mon-bucket-jeremy-session-validation/hotel.csv')

In [17]:
# engine = create_engine(f"postgresql+psycopg2://{YOUR_USERNAME}:{YOUR_PASSWORD}@{YOUR_HOSTNAME}/postgres", echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)


In [18]:
df.to_sql("hotel", engine)

2022-12-04 12:10:43,088 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hotel")
2022-12-04 12:10:43,088 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-04 12:10:43,089 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hotel")
2022-12-04 12:10:43,089 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-04 12:10:43,091 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-04 12:10:43,092 INFO sqlalchemy.engine.Engine 
CREATE TABLE hotel (
	"index" BIGINT, 
	"Unnamed: 0" BIGINT, 
	ville_rechercher TEXT, 
	"ville_précise" TEXT, 
	hotel TEXT, 
	url TEXT, 
	note TEXT, 
	description TEXT, 
	latitutde FLOAT, 
	longitude FLOAT, 
	"2022-12-04 15:00:00" TEXT, 
	"2022-12-05 15:00:00" TEXT, 
	"2022-12-06 15:00:00" TEXT, 
	"2022-12-07 15:00:00" TEXT, 
	"2022-12-08 15:00:00" TEXT
)


2022-12-04 12:10:43,092 INFO sqlalchemy.engine.Engine [no key 0.00038s] ()
2022-12-04 12:10:43,093 INFO sqlalchemy.engine.Engine CREATE INDEX ix_hotel_index ON hotel ("index")
2022-12-04 12:10:43,093 INFO sql

878

In [19]:
conn = engine.connect()

In [21]:
statement = text("SELECT DISTINCT ville_précise FROM hotel")

In [22]:
result = conn.execute(statement)

2022-12-04 12:12:18,073 INFO sqlalchemy.engine.Engine SELECT DISTINCT ville_précise FROM hotel
2022-12-04 12:12:18,073 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ()


In [23]:
result.fetchall()

[('Le Mont-Saint-Michel',),
 ('Ardevon',),
 ('Pontorson',),
 ('Beauvoir',),
 ('Saint-Malo',),
 ('Paramé, Saint-Malo',),
 ('Intra muros, Saint-Malo',),
 ('Saint-Servan, Saint-Malo',),
 ('Sillon, Saint-Malo',),
 ('Bayeux',),
 ('Centre-ville de Rouen, Rouen',),
 ('Rouen',),
 ('Perret, Le Havre',),
 ('Le Havre',),
 ('6e arr., Paris',),
 ('13e arr., Paris',),
 ('9e arr., Paris',),
 ('5e arr., Paris',),
 ('10e arr., Paris',),
 ('17e arr., Paris',),
 ('14e arr., Paris',),
 ('16e arr., Paris',),
 ('3e arr., Paris',),
 ('15e arr., Paris',),
 ('11e arr., Paris',),
 ('8e arr., Paris',),
 ('19e arr., Paris',),
 ('2e arr., Paris',),
 ('12e arr., Paris',),
 ('Amiens',),
 ('Centre de Lille, Lille',),
 ('Lille Sud, Lille',),
 ('Vieux-Lille, Lille',),
 ('Vauban, Lille',),
 ('Lille',),
 ('Faubourg de Béthune, Lille',),
 ('Wazemmes, Lille',),
 ('Fives, Lille',),
 ('Centre de Strasbourg - Petite France - Cathédrale, Strasbourg',),
 ('Strasbourg',),
 ('Quartier Européen - Orangerie, Strasbourg',),
 ('Neudo